In [44]:
import numpy as np
import random
import math
import re
import matplotlib.pyplot as plt
 
def get_neighborhood(center, radix, domain):
    #radix尺度参数至少为1，用于固定方差
    if radix < 1:
        radix = 1
    #计算序列数值上差值delta
    deltas = np.absolute(center - np.arange(domain))
    #这边设置了一个堆成的距离修正，使得距离最大不超过数目的一半，可能是为了增强学习的强度
    distances = np.minimum(deltas, domain - deltas)
    #返回权值
    return np.exp(-(distances*distances) / (2*(radix*radix)))
 
def main(data_path,save_path):
    #读入数据，注意这里的data_path是文件的直接定位路径
    with open(data_path, 'r', encoding='UTF-8') as f:
        lines = f.readlines()
    #将读入的数据存储在citys中
    citys = []
    #学习率设置
    lr = 0.8
    start = 0
    for line in lines[0:-1]:
        start = start + 1
        if line.find('NODE_COORD_SECTION') != -1:
            break
    for line in lines[start:-1]:
        try:
            if line=="\n" or line.find("EOF") != -1:
                continue
            cord = line.replace('\n', '')
            cord = cord.strip()
            cord = re.sub(' +', ' ', cord)
            cord = cord.split(' ')
            citys.append([int(cord[0]), float(cord[1]), float(cord[2])])
        except:
            print(line)
    #神经元的数目设置为城市的八倍
    n = len(citys)*8
    citys = np.array(citys)
    temp_citys = citys.copy()
 
    #normalize，与后续神经元的位置对应
    x_min, y_min = citys.min(axis=0)[1:]
    x_max, y_max = citys.max(axis=0)[1:]
    citys[:,1] = (citys[:,1]-x_min)/(x_max-x_min)
    citys[:,2] = (citys[:,2]-y_min)/(y_max-y_min)
    # for i in range(len(citys)):
    #     citys[i][1], citys[i][2] = (citys[i][1]-min_c)/(max_c-min_c), (citys[i][2]-min_c)/(max_c-min_c)
    #随机生成神经元的初始位置
    network = np.random.rand(n, 2)
    iterations = 100000
    for i in range(iterations):
        #随机选择一个城市
        select_city = random.randint(0,len(citys)-1)
        city = citys[select_city][1:]
 
        #找到与被选中城市距离最小的神经元
        nearest_n = -1
        min_dis = float('inf')
        for j in range(len(network)):
            #计算欧氏距离
            dis = math.sqrt(sum(pow(city - network[j], 2)))
            #更新距离和神经元标记
            if dis < min_dis:
                min_dis = dis
                nearest_n = j
 
        #以该神经元为中心建立高斯分布，即计算权值，使得编号上更接近该神经元的神经元调整更多
        gaussian = get_neighborhood(nearest_n, n // 10, network.shape[0])
        #np.newaxis增加新维度以实现矩阵乘法
        network += gaussian[:, np.newaxis] * lr * (city - network)
        n = n * 0.9997
        lr = lr * 0.99997
 
        if i%1000 == 0:
            plt.scatter(citys[:,1], citys[:,2], color='red', s=4)
            plt.plot(network[:,0], network[:,1], 'r.', ls='-', color='#0063ba', markersize=2)
            plt.savefig(save_path+'iter{}.jpg'.format(i), bbox_inches='tight', pad_inches=0, dpi=200)
            plt.close()
 
 
        if n < 1:
            print('Radius has completely decayed, finishing execution',
            'at {} iterations'.format(i))
            iterations = i
            break
        if lr < 0.001:
            print('Learning rate has completely decayed, finishing execution',
            'at {} iterations'.format(i))
            iterations = i
            break
    #输出break时的训练结果
    plt.scatter(citys[:,1], citys[:,2], color='red', s=4)
    plt.plot(network[:,0], network[:,1], 'r.', ls='-', color='#0063ba', markersize=2)
    plt.savefig(save_path+'/iter{}.jpg'.format(iterations), bbox_inches='tight', pad_inches=0, dpi=200)
    plt.close()

    #储存与citys坐标最近的神经元是哪个
    new_citys = []
    for i in range(len(citys)):
        city = citys[i][1:]
        nearest_city = -1
        min_dis = float('inf')
        for j in range(len(network)):
            dis = math.sqrt(sum(pow(city - network[j], 2)))
            if dis < min_dis:
                min_dis = dis
                nearest_city = j
        new_citys.append([i, nearest_city])
        
    #根据神经元的序列排序,训练后的神经元序列越近距离越近
    new_citys_ = sorted(new_citys, key=lambda x:x[1])
    #添加第一个到最后，使得其能够成环
    new_citys_.append(new_citys_[0])
    new_citys_ = np.array(new_citys_)
    final_path = temp_citys[new_citys_[:,0],:][:,1:]
    path_lenght = 0
    for i in range(len(final_path)-1):
        path_lenght += math.sqrt(sum(pow(final_path[i] - final_path[i+1], 2)))
    print('final distance:{}'.format(path_lenght))
    plt.scatter(final_path[:, 0], final_path[:, 1], color='red', s=4)
    plt.plot(final_path[:, 0], final_path[:, 1], 'r.', ls='-', color='#0063ba', markersize=2)
    plt.savefig(save_path+'route.jpg', bbox_inches='tight', pad_inches=0, dpi=200)
    plt.close()
 
 
 
if __name__ == '__main__':
    #数据集
    data_path = './u1060.tsp'
    #储存路径，注意目录下需要有
    save_path = './result2/'
    main(data_path,save_path)
 

Radius has completely decayed, finishing execution at 30147 iterations
final distance:293196.18831017416
